In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import keras
import rpy2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


HERE WE RUN TWO MAJOR THINGS
COLLABRATIVE FILTERING -PARTIAL SUCCESS BECAUSE SOME TIMES COSINE SIMILARITY BETWEEN USERS =0 AND HENCE WE HAVE TO GO BACK TO PERSONA
CONTENT FILTERING -FULL SUCCESS 

In [ ]:

DF_Rec=pd.read_csv('/content/drive/My Drive/Project Text USML Part 2/Recommendation_ADHD.csv')
DF_Rec2=DF_Rec.iloc[:,:]
DF_Rec2
Y=pd.factorize(DF_Rec2.Drug)
DF_Rec2['DrugIndex']=Y[0]
DF_Rec2.rename(columns={'Unnamed: 0':'User_Index'})
DF_Rec2.columns

Index(['Unnamed: 0', 'Persona_User_t', 'Y_pred_linreg', 'Drug', 'DrugIndex'], dtype='object')

In [ ]:
#Lets rename the first column
mapping = {DF_Rec2.columns[0]:'user-id'}
DF_Rec2 = DF_Rec2.rename(columns=mapping)
DF_Rec2

,user-id,Persona_User_t,Y_pred_linreg,Drug,DrugIndex
0,0,1,7.408572,Guanfacine,0
1,1,2,7.376918,Daytrana,1
2,2,1,7.143011,Intuniv,2
3,3,1,7.364832,Focalin,3
4,4,1,7.348754,Vyvanse,4
...,...,...,...,...,...
295,295,1,7.045653,Amphetamine / dextroamphetamine,6
296,296,1,7.596993,Daytrana,1
297,297,1,7.561470,Bupropion,15
298,298,2,7.392651,Bupropion,15


In [ ]:


!pip install surprise
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import KNNBasic,KNNWithMeans,KNNBaseline
from surprise import accuracy
reader = Reader()

data_table = pd.pivot_table(DF_Rec2,values='Y_pred_linreg',columns='Drug',index='Persona_User_t')
data_table.head()


#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = DF_Rec2.copy()
y = DF_Rec2['DrugIndex']

#Split into training and test datasets, stratified drugs
y

#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


r_matrix = DF_Rec2.pivot_table(values='Y_pred_linreg', index='user-id', columns='DrugIndex')
r_matrix

def cf_user_mean(DrugIndex):
    
    #Check if DrugIndex  exists in r_matrix
    if DrugIndex in r_matrix:
        #Compute the mean of all the ratings given to the drug
        mean_rating = r_matrix[DrugIndex].mean()
    else:
        #Default to a rating of 3.0 in the absence of any information
        mean_rating = 5.0
    
    return mean_rating


r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy

from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity matrix using the dummy ratings for different drugs by different users 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)


cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim



X=DF_Rec2[DF_Rec2.Persona_User_t==4]
min(X.Y_pred_linreg)

6.987127909133048

Collabrative Filtering. We cannot do item based filtering as each user takes only 1 medication. Since a user takes only 1 medication ; the rating vector for a given user will contain only one item.

Next we will take collabrative filtering 

In [ ]:
r_matrix = DF_Rec2.pivot_table(values='Y_pred_linreg', index='user-id', columns='DrugIndex')
r_matrix

DrugIndex,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
user-id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7.408572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,7.376918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,7.143011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,7.364832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,7.348754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,NaN,NaN,NaN,7.045653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,NaN,7.596993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.561470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
item_Matrix=r_matrix.T# Transpose the matrix where each drug /item becomes observations and each user becomes 
cosine_sim = pd.DataFrame(cosine_sim, index=item_Matrix.index, columns=item_Matrix.index)
cosine_sim
#Drug Similarity Cosine 

DrugIndex,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
DrugIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)
cosine_sim
#User Similarity 

user-id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
user-id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

!pip install surprise
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import KNNBasic,KNNWithMeans,KNNBaseline
from surprise import accuracy
reader = Reader()

data_table = pd.pivot_table(DF_Rec2,values='Y_pred_linreg',columns='Drug',index='Persona_User_t')
data_table.head()
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = DF_Rec2.copy()
y = DF_Rec2['DrugIndex']



0       0
1       1
2       2
3       3
4       4
       ..
295     6
296     1
297    15
298    15
299     2
Name: DrugIndex, Length: 300, dtype: int64

In [ ]:
data_table.head()

Drug,Adderall,Adderall XR,Adzenys XR-ODT,Amphetamine,Amphetamine / dextroamphetamine,Atomoxetine,Bupropion,Clonidine,Concerta,Daytrana,Desoxyn,Dexedrine,Dexmethylphenidate,Dextroamphetamine,Dyanavel XR,Focalin,Focalin XR,Guanfacine,Intuniv,Kapvay,Lisdexamfetamine,Methylin ER,Methylphenidate,Mydayis,Nuvigil,Omega-3 polyunsaturated fatty acids,Pemoline,Quillivant XR,Ritalin,Strattera,Vyvanse,Wellbutrin SR,Zenzedi
Persona_User_t,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,7.431250,7.378587,7.399039,7.465148,7.371008,7.340242,7.476755,7.399793,7.408455,7.292292,7.254403,7.736229,7.273189,7.228844,7.525824,7.475875,7.391042,7.262051,7.331959,7.08326,7.356852,7.232881,7.358674,7.375689,7.506798,7.574061,7.48591,7.180260,7.320291,7.331914,7.405722,7.027213,7.349151
2,7.196035,7.365995,NaN,NaN,7.295414,7.560860,7.392651,NaN,7.395327,7.376918,NaN,NaN,7.063179,NaN,NaN,NaN,7.281638,NaN,NaN,NaN,8.010493,NaN,7.384436,NaN,NaN,NaN,NaN,7.076548,NaN,7.156719,7.573117,NaN,NaN
4,NaN,7.473686,NaN,NaN,NaN,7.549819,NaN,NaN,7.309163,NaN,NaN,7.695934,NaN,NaN,NaN,NaN,7.305269,NaN,NaN,NaN,6.987128,NaN,7.420696,NaN,NaN,NaN,NaN,NaN,NaN,7.483017,7.229542,NaN,NaN


In [ ]:

#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def score(cf_model):
    
    #Construct a list of user-drug tuples from the testing dataset
    id_pairs = zip(X_test['Unnamed: 0'], X_test['drug_id'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(user, medicine) for (user, medicine) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['rating'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)


r_matrix = DF_Rec2.pivot_table(values='Y_pred_linreg', index='user-id', columns='DrugIndex')
r_matrix

#Matrix of Drug Index Vs User ID

DrugIndex,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
user-id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7.408572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,7.376918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,7.143011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,7.364832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,7.348754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,NaN,NaN,NaN,7.045653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,NaN,7.596993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.561470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sub_data=DF_Rec2.iloc[:,[0,2,4]]
sub_data

,user-id,Y_pred_linreg,DrugIndex
0,0,7.408572,0
1,1,7.376918,1
2,2,7.143011,2
3,3,7.364832,3
4,4,7.348754,4
...,...,...,...
295,295,7.045653,6
296,296,7.596993,1
297,297,7.561470,15
298,298,7.392651,15


In [ ]:
reader = Reader(rating_scale=(0, 9))
data = Dataset.load_from_df(sub_data, reader)
data

In [ ]:
X2=DF_Rec2.loc[DF_Rec2['Y_pred_linreg'] == p]# Finding the user id with lowest score  
X2
#


,user-id,Persona_User_t,Y_pred_linreg,Drug,DrugIndex
250,250,4,6.987128,Lisdexamfetamine,5


In [ ]:
def cf_user_wmean(df,user_id,DrugIndex):
    DF_Rec2=df
        #existing rating
    fildf=DF_Rec2.loc[DF_Rec2['user-id'] == user_id]# put the filtered DF out
    current=fildf.Y_pred_linreg.values[0]
    #Check if drug_id exists in r_matrix
    if DrugIndex in r_matrix:
        #existing rating
        #Get the similarity scores for the user in question with every other user
        sim_scores0 = cosine_sim[user_id] #Cosine Similarity for all Users compared to given user which is like a vector 
        sim_scores0_array=sim_scores0.values
        #Get the user rating for drug
        m_ratings = r_matrix[DrugIndex]# Rating Vector for Given Drug 
        #Extract the indices containing NaN in the Rating 
        idx = m_ratings[m_ratings.isnull()].index
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()  
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores3 = sim_scores0.drop(idx)
        is_all_zero = np.all((sim_scores3.values == 0))
        if (is_all_zero==True):
              print("No similar user could be computed based on same drug and hence no weightage rating")
              y=np.argwhere(sim_scores0.values>0.5)
              for i in range(len(y)):
                    M=DF_Rec2.loc[DF_Rec2['user-id']==y[i][0]]
                    if(M.Persona_User_t.values[0]==4 or M.Persona_User_t.values[0]==3)&(M.Y_pred_linreg.values[0]>current):
                      z=M.DrugIndex.values[0]
                      print("Drug id aligning to Persona and cosine similarity that could benefit him",z)
                    else:
                      if (M.Y_pred_linreg.values[0]>current):
                        z=M.DrugIndex.values[0]
                        print(" Drug id agnostic to persona that could benefit",z)
              return z
        else:
          #Compute the final weighted mean
            wmean_rating = np.dot(sim_scores3, m_ratings)/ sim_scores3.sum()
            print("Weighted rating of the drug against this person is",wmean_rating)
    else:
        #Default to a rating of 3.0 in the absence of any information
        print("New drug and default rating is ", wmean_rating=5.0)
    
    return wmean_rating


In [ ]:
cf_user_wmean(DF_Rec2,user_id=105,DrugIndex=1)

Weighted rating of the drug against this person is nan


nan

In [ ]:
cf_user_wmean(DF_Rec2,user_id=250,DrugIndex=1)

Weighted rating of the drug against this person is nan


nan

In [ ]:
def cf_drug_wmean(user_id,DrugIndex):
    #Check if drug_id exists in r_matrix
    if user_id in r_matrix:
        #Get the similarity scores between drugs
        sim_scores = cosine_sim[DrugIndex] 
        #Get the user rating for drug
        m_ratings = r_matrix[user_id]# Rating Vector for Given user
        #Extract the indices containing NaN in the Rating 
        idx = m_ratings[m_ratings.isnull()].index
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        #Compute the final weighted mean
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        wmean_rating = 3.0
    
    return wmean_rating


In [ ]:
cf_drug_wmean(user_id=10,DrugIndex=5)
#As expected it will not work because 1 user only ranks 1 drug and hence every there is no similar items between drugs and hence it  is as expected 

nan

In [ ]:
X=np.array([1,2,3,4])
X
np.delete(X,[0,1],axis=0)

array([3, 4])

Now lets use content based filtering 

In [ ]:
#Let X be a matrix whose rows represent features of users 
X = pd.get_dummies(DF_Rec2.Persona_User_t)
X_a=X.values
m.shape
#sample patient of Persona 4 who has low rating 
test=X.iloc[289,:]# Persona 4 
Xa= np.append(X_a,np.ones([X_a.shape[0],1]),1)
Xa

array([[1., 0., 0., 1.],
       [0., 1., 0., 1.],
       [1., 0., 0., 1.],
       ...,
       [1., 0., 0., 1.],
       [0., 1., 0., 1.],
       [1., 0., 0., 1.]])

Test Patient 

In [ ]:
test=X.iloc[289,:]# Persona 4 
test#using hot encoding of 3 features of Persona 4 

1    0
2    0
4    1
Name: 289, dtype: uint8

BETA CALCULATION  WE BASICALLY USED LINEAR REGRESSION .

In [ ]:
import numpy as np
def beta_calc(DrugIndex,matrix,X):
  r_matrix=matrix
  if DrugIndex in r_matrix:
    m_ratings = r_matrix[DrugIndex]# Rating vector for given drug among all users
    idx = m_ratings[m_ratings.isnull()].index
    m_ratings = m_ratings.dropna()
    y=m_ratings
    X_arr=X
    Xa=np.delete(X_arr,idx.values,axis=0)
    m=np.dot(Xa.T,Xa)
    R=np.linalg.det(m)
    if (R==0):# CHECK IF MATRIX IS SINGULAR 
       from sklearn.linear_model import LinearRegression
       lr = LinearRegression()
       X2= np.delete(Xa,3, axis=1)
       model=lr.fit(X2,y)
       beta=[lr.coef_,lr.intercept_]

    else:#NON SINGULAR USE ALGEBRA FORMULA 
      minv=np.linalg.matrix_power(m,-1)
      beta=np.dot(minv,np.dot(Xa.T,y))
    return beta,model
    


In [ ]:
vec_beta,model=beta_calc(14,r_matrix,Xa)
vec_beta
vec_beta
test2=np.array(test)# convert the hot encoding of Persona 4 to array forma t
test3=test2.reshape(1,-1)# reshape it in format that linear regression is possible 
print(test3)
y_predict = model.predict(test3)
y_predict

[[0 0 1]]


array([7.3136424])

USE PREDICTION MODEL OF RATING OF DRUG GIVEN PERSONA FEATURE VALEUS FOR ALL PATIENTS/UNIQUE USERS 


In [ ]:
y_predict_t=model.predict(X.values)
from sklearn.metrics import mean_squared_error
train_error=mean_squared_error(y_predict_t,DF_Rec2.Y_pred_linreg)
train=train_error*100
print(train)

4.354370784641087


Hence based on the above Drug Index 10; Foxaline XR can be provided to this person as he has persona 4 

In [ ]:
r_matrix.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
           dtype='int64', name='DrugIndex')

RUNNING PREDICTION FOR ALL DRUGS  GIVEN A PERSONA  USING FOR LOOP

In [ ]:
index2=r_matrix.columns.values
#Drug recommendation System 
rating_list=[]
for i in range(0,len(index2)):
  beta_vec,model=beta_calc(i,r_matrix,Xa)#beta calculation 
  rating_persona4=model.predict(test3)
  rating_list.append(rating_persona4[0])
rating_list

[7.262050578293465,
 7.334604964161771,
 7.331959063446128,
 7.475875189322946,
 7.229542409156022,
 6.987127909133048,
 7.333211018512324,
 7.5498187876628915,
 7.309162787085489,
 7.483016897165965,
 7.305268830521839,
 7.420695651289455,
 7.320291046966427,
 7.083260069301423,
 7.313642395359589,
 7.434702700959136,
 7.168183826625554,
 7.0272127487051455,
 7.349151015386438,
 7.254402912252558,
 7.473685905625978,
 7.399039300162935,
 7.485909631857249,
 7.50679788482836,
 7.525823960765995,
 7.228843712119989,
 7.232881196348145,
 7.695933798870403,
 7.4651483393795415,
 7.128404105794896,
 7.574061243378503,
 7.3756885716758225,
 7.399792652295308]

Recommended medicine as per rating will be the best rated Medicine for Person

In [ ]:
index=np.argmax(rating_list)
Output=pd.DataFrame(DF_Rec2.loc[DF_Rec2['DrugIndex'] == index].Drug)
Output.iloc[0,:]

Drug    Dexedrine
Name: 140, dtype: object

The above drug  seems the best drug for the person who " Lisdexamfetamine".  Hence our system recommend Doctor  to take Dexedrine if he does not have allergic reaction